In [ ]:
import pandas as pd
import os
import numpy as np
from bp import BP, parse_newick, write_newick
import glob
from biom.util import biom_open
from biom import Table
from qiime2 import Metadata, Artifact

- Create biom table and taxonomy for agp and emp samples

In [ ]:
base_dir = '2023.05.22-decomp-asvs'
emp_agp_meta = pd.read_csv(f'{base_dir}/sample-metadata.tsv', sep='\t', index_col=0)
emp_agp_meta['both'] = emp_agp_meta['is_emp'] & emp_agp_meta['is_agp']

decomp_asvs = {
    file.split('/')[2] 
    for file in glob.glob(f'{base_dir}/results/*') if file != '.DS_Store' 
}

with open('agp_emp_samples.txt', 'w') as f:
    f.write('\n'.join(emp_agp_meta.index))
    f.write('\n')
    
!redbiom fetch samples \
--from agp_emp_samples.txt \
--context Deblur_2021.09-Illumina-16S-V4-150nt-ac8c0b \
--output all_decomp_agp_emp.biom \
--resolve-ambiguities 'most-reads'

# get taxa of all agp-emp samples
!qiime tools import \
--type "FeatureTable[Frequency]" \
--input-path all_decomp_agp_emp.biom \
--output-path all_decomp_agp_emp.biom.qza

!qiime greengenes2 taxonomy-from-table \
--i-reference-taxonomy 2022.10.taxonomy.asv.nwk.qza \
--i-table all_decomp_agp_emp.biom.qza \
--o-classification all_decomp_agp_emp-tax.qza

all_decomp_agp_emp_tax = Artifact.load('all_decomp_agp_emp-tax.qza').view(pd.DataFrame)
decomp_tax = pd.read_csv('decom-tax.tsv', sep='\t', index_col=0)

with biom_open('all_decomp_agp_emp.biom') as f:
    table = Table.from_hdf5(f)
table

- keep asvs with read count > 100

In [ ]:
table.transform(lambda data, id_, meta: np.where(data > 100, data, 0), inplace=True)
table.pa(inplace=True)
asv_agp_emp = table.ids(axis='observation')
table

- create table per environment type

In [ ]:
def get_samples(emp_agp_meta, env, agp_emp):
    return emp_agp_meta.loc[
        (emp_agp_meta[agp_emp]) &
        (emp_agp_meta.env_package == env) &
        (emp_agp_meta.index.isin(table.ids(axis='sample')))
    ]

agp_gut_samp = get_samples(emp_agp_meta, 'human-gut', 'is_agp')
gut_table = table.filter(agp_gut_samp.index, axis='sample', inplace=False)

agp_skin_samp = get_samples(emp_agp_meta, 'human-skin', 'is_agp')
skin_table = table.filter(agp_skin_samp.index, axis='sample', inplace=False)

emp_soil_samp = get_samples(emp_agp_meta, 'soil', 'is_emp')
soil_table = table.filter(emp_soil_samp.index, axis='sample', inplace=False)

emp_host_samp = get_samples(emp_agp_meta, 'host-associated', 'is_emp')
host_table = table.filter(emp_host_samp.index, axis='sample', inplace=False)

- get most prevelent asvs per environment 

In [ ]:
def get_most_abundant(t, n=50):
    return np.argsort(t.sum(axis='observation'))[-n:]
top_gut = asv_agp_emp[get_most_abundant(gut_table)]
top_skin = asv_agp_emp[get_most_abundant(skin_table)]
top_soil = asv_agp_emp[get_most_abundant(soil_table)]
top_host = asv_agp_emp[get_most_abundant(host_table)]
top_asvs = np.concatenate((top_gut, top_skin, top_soil, top_host), axis=None)

- create metadata

In [ ]:
keep = set(top_asvs).union(set(decomp_asvs))
table.filter(top_asvs, axis='observation', inplace=True)
gut_table = gut_table.filter(table.ids(axis='observation'), axis='observation', inplace=False)
skin_table = skin_table.filter(table.ids(axis='observation'), axis='observation', inplace=False)
soil_table = soil_table.filter(table.ids(axis='observation'), axis='observation', inplace=False)
host_table = host_table.filter(table.ids(axis='observation'), axis='observation', inplace=False)

with open('all-top-counts.tsv', 'w') as f:
    f.write('feature id\tgut\tskin\tsoil\thost\tis_decomp\n')
    for ob, gut, skin, soil, host in zip(
        table.ids(axis='observation'),
        gut_table.sum(axis='observation'),
        skin_table.sum(axis='observation'),
        soil_table.sum(axis='observation'),
        host_table.sum(axis='observation')
    ):
        f.write(f'{ob}\t{gut}\t{skin}\t{soil}\t{host}\t{1 if ob in set(decomp_asvs) else 0}\n')
    missing_decomp = [asv for asv in set(decomp_asvs) if asv not in table.ids(axis='observation')]
    for asv in missing_decomp:
        f.write(f'{asv}\t{0}\t{0}\t{0}\t{0}\t{1}\n')
        
tax_meta = all_decomp_agp_emp_tax.loc[all_decomp_agp_emp_tax.index.isin(set(table.ids(axis='observation')).union(set(decomp_asvs)))]
tax_meta.to_csv('top_tax.tsv', sep='\t', index=True, index_label='feature id')

meta = pd.read_csv('all-top-counts.tsv', sep='\t', index_col=0)
for env in ['gut', 'soil', 'skin','host']:
    meta[env] = meta[env].rank(method='min')
    
meta.to_csv('top-rank-tax.tsv', index=True, index_label='feature id', sep='\t')

dec_tax = pd.read_csv('decom-tax.tsv', sep='\t', index_col=0)
tax = pd.read_csv('top_tax.tsv', sep='\t', index_col=0)
pd.concat((tax, dec_tax.loc[~(dec_tax.index.isin(tax.index))])).to_csv('top-rank-tax.tsv', index=True, index_label='feature id', sep='\t')

meta = pd.read_csv('all-top-counts.tsv', index_col=0, sep='\t')

tax = pd.read_csv('top-rank-tax.tsv', index_col=0, sep='\t')
meta = pd.read_csv('all-top-counts.tsv', index_col=0, sep='\t')
meta['Taxon'] = tax.loc[tax.index.isin(meta.index), 'Taxon']

meta.to_csv('top-ranks-meta-w-tax.tsv', sep='\t', index=True, index_label='feature id')

- create tree

In [ ]:
with open('2022.10.phylogeny.asv.nwk', 'r') as f:
    tree = parse_newick(f.readline())
tree = tree.shear(set(meta.index.to_list()))

with open('tree-top-asvs-decomp.nwk', 'w') as f:
    write_newick(tree, f, True)
!qiime tools import --type "Phylogeny[Rooted]" --input-path tree-top-asvs-decomp.nwk --output-path tree-top-asvs-decomp.nwk.qza

In [ ]:
!qiime empress tree-plot \
--i-tree tree-top-asvs-decomp.nwk.qza \
--m-feature-metadata-file top-ranks-meta.tsv \
--o-visualization rank-plot